In [1]:
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
# add MAE
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
power = pd.read_csv('debug_logs/real_power_gurobi_phase_3_step_leap_1_forecast_step_1.csv')
power = power.melt(id_vars=['time_step'], var_name='building', value_name='net_target')
power = power.sort_values(by=['time_step', 'building'])
power['building'] = power['building'].str[-1].astype(int)
power = power[power['time_step'] != -1]
lead_times = [f'+{i}h' for i in range(24)]
# upload min-max scaler from scalers folders with .save extension
power_norm = pd.DataFrame()
for building in range(7):
    power_norm_temp = pd.DataFrame()
    power_norm_temp['time_step'] = power.loc[power['building'] == building, 'time_step']
    power_norm_temp['building'] = building
    power_norm_temp['net_target'] = power.loc[power['building'] == building, 'net_target']
    for i, lead_time in enumerate(lead_times):
        power_norm_temp[lead_time] = power_norm_temp['net_target'].shift(-int(i))
    power_norm = pd.concat([power_norm, power_norm_temp], axis=0)
power_norm = power_norm.dropna()
power_norm = power_norm.sort_values(by=['time_step', 'building'])
# drop the net_target column
power_norm = power_norm.drop(columns=['net_target'])
# set the index to time_step and building
power_norm = power_norm.set_index(['time_step', 'building'])
power_norm

+0h           +1h           +2h           +3h  \
time_step building                                                           
0         0         6.907833e-01  6.887500e-01  6.335667e-01  6.768111e-01   
          1         1.000000e-07  1.000000e-07  1.000000e-07  1.000000e-07   
          2         1.405750e+00  1.187467e+00  1.100133e+00  1.247567e+00   
          3         3.328667e-01  7.353000e-01  2.221967e+00  8.956500e-01   
          4         1.000000e-07  1.000000e-07  1.000000e-07  1.000000e-07   
...                          ...           ...           ...           ...   
8762      2         6.969639e-01  3.210000e-01  3.210944e-01  3.088667e-01   
          3         4.002333e-01  3.982500e-01  3.777389e-01  6.308833e-01   
          4         1.000000e-07  1.000000e-07  1.000000e-07  1.000000e-07   
          5         1.114000e+00  1.494167e+00  1.409217e+00  1.421167e+00   
          6         1.227383e+00  1.466233e+00  3.340667e-01  3.619333e-01   

                             +4h           +5h       +6h       +7h       +8h  \
time_step building                                                             
0         0         6.204500e-01  1.463322e+00  1.589600  0.230944  0.224400   
          1         1.000000e-07 -4.568323e-02  0.074033  0.331600  0.985483   
          2         7.937833e-01  9.422500e-01  1.120967  1.330017  0.269967   
          3         3.305833e-01  4.994500e-01  3.203350 -0.007133 -0.404317   
          4         1.000000e-07  1.000000e-07  0.069367  0.495367  1.215700   
...                          ...           ...       ...       ...       ...   
8762      2         3.979000e-01  4.443667e-01  0.261883  0.903417 -0.208967   
          3         2.034978e+00  4.323000e-01  0.202372 -0.081250 -0.404711   
          4         1.000000e-07  1.000000e-07 -0.080200  0.353133  1.000117   
          5         1.377900e+00  1.404483e+00  1.528767  1.095467  0.279600   
          6         3.675000e-01  1.177217e+00  1.352033 -0.945517 -1.489833   

                         +9h  ...      +14h      +15h      +16h      +17h  \
time_step building            ...                                           
0         0        -0.346778  ... -0.517433 -0.148833  0.445283  1.518111   
          1         1.902900  ... -0.023517 -0.355717 -0.244867 -0.174317   
          2         0.224567  ...  0.305383 -0.234767  0.502983  0.846617   
          3        -0.658783  ... -0.111100 -0.087400 -0.441150  0.256567   
          4         1.586317  ...  0.246533  0.060083  0.088150  0.002800   
...                      ...  ...       ...       ...       ...       ...   
8762      2         0.269117  ... -1.717206  0.196217  1.476428  2.158550   
          3        -0.662000  ...  1.747511  1.755267  2.693389  3.650067   
          4         1.832300  ...  3.023450  2.676717  2.130667  1.408808   
          5         0.487600  ... -1.725900  0.125133  0.400967 -0.025800   
          6        -2.205650  ...  0.941767 -1.639083  1.222650  2.820967   

                            +18h          +19h          +20h          +21h  \
time_step building                                                           
0         0         2.988317e+00  3.048861e+00  3.568817e+00  2.022833e+00   
          1        -1.911658e-02  1.000000e-07  1.000000e-07  1.000000e-07   
          2         2.221517e+00  1.808833e+00  2.170200e+00  1.665800e+00   
          3         2.051300e+00  4.820833e-01  8.071167e-01  4.758667e-01   
          4         1.000000e-07  1.000000e-07  1.000000e-07  1.000000e-07   
...                          ...           ...           ...           ...   
8762      2         1.526644e+00  3.313683e+00  1.880772e+00  3.529250e+00   
          3         3.671528e+00  2.727883e+00  1.323300e+00  1.507017e+00   
          4         5.775250e-01 -3.678333e-02  1.000000e-07  1.000000e-07   
          5         3.426450e+00  1.699617e+00  1.798450e+00  1.804633e+00   
          6         3.207700e+00  3

In [7]:
power_norm.reset_index()

,time_step,building,+0h,+1h,+2h,+3h,+4h,+5h,+6h,+7h,...,+15h,+16h,+17h,+18h,+19h,+20h,+21h,+22h,+23h,scenario
0,0,0,6.907833e-01,6.887500e-01,6.335667e-01,6.768111e-01,6.204500e-01,1.463322e+00,1.589600,0.230944,...,-0.148833,0.445283,1.518111,2.988317e+00,3.048861e+00,3.568817e+00,2.022833e+00,2.498350e+00,8.819222e-01,0
1,0,1,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,-4.568323e-02,0.074033,0.331600,...,-0.355717,-0.244867,-0.174317,-1.911658e-02,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0
2,0,2,1.405750e+00,1.187467e+00,1.100133e+00,1.247567e+00,7.937833e-01,9.422500e-01,1.120967,1.330017,...,-0.234767,0.502983,0.846617,2.221517e+00,1.808833e+00,2.170200e+00,1.665800e+00,1.178150e+00,1.053017e+00,0
3,0,3,3.328667e-01,7.353000e-01,2.221967e+00,8.956500e-01,3.305833e-01,4.994500e-01,3.203350,-0.007133,...,-0.087400,-0.441150,0.256567,2.051300e+00,4.820833e-01,8.071167e-01,4.758667e-01,5.136000e-01,7.085167e-01,0
4,0,4,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0.069367,0.495367,...,0.060083,0.088150,0.002800,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61336,8762,2,6.969639e-01,3.210000e-01,3.210944e-01,3.088667e-01,3.979000e-01,4.443667e-01,0.261883,0.903417,...,0.196217,1.476428,2.158550,1.526644e+00,3.313683e+00,1.880772e+00,3.529250e+00,6.526778e-01,1.780217e+00,0
61337,8762,3,4.002333e-01,3.982500e-01,3.777389e-01,6.308833e-01,2.034978e+00,4.323000e-01,0.202372,-0.081250,...,1.755267,2.693389,3.650067,3.671528e+00,2.727883e+00,1.323300e+00,1.507017e+00,5.051389e+00,3.822333e-01,0
61338,8762,4,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,-0.080200,0.353133,...,2.676717,2.130667,1.408808,5.775250e-01,-3.678333e-02,1.000000e-07,1.000000e-07,1.000000e-07,9.833333e-08,0
61339,8762,5,1.114000e+00,1.494167e+00,1.409217e+00,1.421167e+00,1.377900e+00,1.404483e+00,1.528767,1.095467,...,0.125133,0.400967,-0.025800,3.426450e+00,1.699617e+00,1.798450e+00,1.804633e+00,1.343483e+00,8.525833e-01,0


In [4]:
# add a column scenario with 0 in all rows
power_norm['scenario'] = 0
# bring the scenario column as the second column
power_norm 

In [5]:
power_norm.to_csv('data/citylearn_challenge_2022_phase_3/perfect_forecast.csv')